# Structured Output with OpenAI
In order to build reliable pipelines in which LLMs consistently return output in the same format, we are using a **Structured Output**
- This means that we define a blueprint for the output
- We pass the 'blueprint' to the LLM
- Then the LLM output will conform to the blueprint.

This 'blueprint' in the LLM jargon is often called a "schema".

In [ ]:
# openai for LLM, pydantic to define the schema
!pip install openai pydantic --quiet

In [ ]:
from pydantic import BaseModel
from openai import OpenAI
from google.colab import userdata

In [ ]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

## Getting started with Structured Generation
Let's imagine we want to consistently generate RPG characters with:
- a name
- an age
- a city
- a profession
- a background story
- inventory

We'll define the schema (the blueprint) for the structured output.

In [ ]:
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
from pydantic import BaseModel
from enum import Enum

# Here we want to limit the choices to some pre-defined cities
class City(str, Enum):
    aria = "Aria"
    kniga = "Kniga"
    aquabah = "Aquabah"
    torini = "Torini"

# here we'll let the AI make up the rest
class Character(BaseModel):
    name: str
    age: int
    city: City
    job: str
    two_sentences_background_story: str
    inventory: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful table-top RPG gamemaster assistant."},
        {
            "role": "user",
            "content": "Generate a character for a low-fantasy RPG campaign. Be creative"},
    ],
    temperature=1.0,
    response_format=Character,
)

In [ ]:
message = completion.choices[0].message

In [ ]:
print(f"Name: {message.parsed.name}")
print(f"age: {message.parsed.age}")
print(f"City: {message.parsed.city}")

print(f"Job: {message.parsed.job}")
print(f"Inventory: {message.parsed.inventory}\n")

Name: Elysia Thornwood
age: 27
City: Torini
Job: Herbalist
Inventory: ["Herbalist's kit", 'Healing potions', 'Dried herbs', 'Forest map', 'Sturdy sling', 'Notebook of remedies']



In [ ]:
message.parsed.two_sentences_background_story

"Elysia grew up in the tranquil forests surrounding Torini, learning the secrets of the plants from her grandmother, a renowned herbalist. After her grandmother's mysterious disappearance, Elysia now travels the realm seeking knowledge and allies to uncover the truth behind her family's legacy."

# Structured Output with Instructor

`instructor` is a popular library for structured outputs powered by llms. Designed for simplicity, transparency, and control.

It also used pydantic so you'll see it is very similar.

In [ ]:
!pip install instructor --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.1 MB/s eta 0:00:00


In [ ]:
import instructor

# Patch the OpenAI client
client = instructor.from_openai(
    OpenAI(
        api_key=userdata.get('OPENAI_API_KEY')
        )
    )

# Generate structured data from natural language
character = client.chat.completions.create(
    model="gpt-4o-mini",
    response_model=Character,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful table-top RPG gamemaster assistant."},
        {
            "role": "user",
            "content": "Generate a Warrior character for a low-fantasy RPG campaign. Be creative"},
    ],
    temperature=1.5
)

In [ ]:
character.name

'Ragnar Boldblade'

In [ ]:
character.age

30

In [ ]:
character.city

<City.torini: 'Torini'>

In [ ]:
character.job

'Warrior'

In [ ]:
character.two_sentences_background_story

"Once a blacksmith's apprentice in Torini, Ragnar discovered his calling on the battlefield after defending his village against a band of marauders. Now, he wanders the land seeking to prove his strength and protect the innocent, wielding his masterfully crafted sword with fearless precision."

Learn more about instructor here: https://python.useinstructor.com/